Plan d’organisation pour la Partie B, inspiré de la Partie A, pour entraîner un classifieur de priorité avec BARThez.  


---

## 1. Membres de l’équipe
   
- Demba Seck secd1700  
- Cheikh Tidiane Gueye 
- Fatou Bintou Boye boyf7012 


---

## 2. Télécharger et préparer le jeu de données


In [2]:
import urllib.request

url = 'https://drive.google.com/uc?export=download&id=13ZfF8DjSvqPJkrY93GJTK2l14IwLVLZ0'
urllib.request.urlretrieve(url, 'IT_Support_Tickets_FR_200.csv')
print('Téléchargement terminé !')

Téléchargement terminé !




---

## 3. Charger le DataFrame et afficher les colonnes


In [3]:
import pandas as pd
df = pd.read_csv("IT_Support_Tickets_FR_200.csv")
print(df.columns)
df

Index(['ID Ticket', 'Date', 'Client', 'Description', 'Type de problème',
       'Priorité'],
      dtype='object')


,ID Ticket,Date,Client,Description,Type de problème,Priorité
0,1,2023-10-01,Service Finances,L'ordinateur ne s'allume pas après une mise à ...,Matériel,Élevée
1,2,2023-10-01,Marketing,"Impossible d'accéder à Outlook - erreur ""Compt...",Compte/Mot de passe,Moyenne
2,3,2023-10-02,RH,Le logiciel SAP plante fréquemment lors de la ...,Logiciel,Élevée
3,4,2023-10-02,IT Support,Connexion Wi-Fi lente dans le bâtiment B,Réseau,Moyenne
4,5,2023-10-03,Direction,Impossible d'imprimer depuis le nouveau copieur,Matériel,Élevée
...,...,...,...,...,...,...
195,196,2024-01-06,Comptabilité,Problème de calcul des amortissements,Logiciel,Élevée
196,197,2024-01-07,Ventes,Problème de tactile sur l'écran interactif,Matériel,Moyenne
197,198,2024-01-07,Service Client,Demande de configuration des signatures groupées,Compte/Mot de passe,Basse
198,199,2024-01-08,Logistique,Problème de synchronisation des inventaires,Logiciel,Moyenne




---

## 4. Préparer les données pour la classification de priorité


In [4]:
# On retire les colonnes inutiles
colonnes_a_retirer = ['ID Ticket', 'Date', 'Client', 'Type de problème']
df_priorite = df.drop(columns=colonnes_a_retirer)
df_priorite.head()

,Description,Priorité
0,L'ordinateur ne s'allume pas après une mise à ...,Élevée
1,"Impossible d'accéder à Outlook - erreur ""Compt...",Moyenne
2,Le logiciel SAP plante fréquemment lors de la ...,Élevée
3,Connexion Wi-Fi lente dans le bâtiment B,Moyenne
4,Impossible d'imprimer depuis le nouveau copieur,Élevée




---

## 5. Encoder la priorité


In [5]:
from sklearn.preprocessing import LabelEncoder

encodeur_priorite = LabelEncoder()
df_priorite['Classe'] = encodeur_priorite.fit_transform(df_priorite['Priorité'])
df_priorite

,Description,Priorité,Classe
0,L'ordinateur ne s'allume pas après une mise à ...,Élevée,2
1,"Impossible d'accéder à Outlook - erreur ""Compt...",Moyenne,1
2,Le logiciel SAP plante fréquemment lors de la ...,Élevée,2
3,Connexion Wi-Fi lente dans le bâtiment B,Moyenne,1
4,Impossible d'imprimer depuis le nouveau copieur,Élevée,2
...,...,...,...
195,Problème de calcul des amortissements,Élevée,2
196,Problème de tactile sur l'écran interactif,Moyenne,1
197,Demande de configuration des signatures groupées,Basse,0
198,Problème de synchronisation des inventaires,Moyenne,1




---

## 6. Split train/val/test


In [6]:
from sklearn.model_selection import train_test_split

X = df_priorite['Description']
y = df_priorite['Classe']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

print(len(X_train), len(X_val), len(X_test))

120 40 40




---

## 7. Tokenizer BARThez


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("moussaKam/barthez")
X_train_list = X_train.tolist()
X_val_list = X_val.tolist()
X_test_list = X_test.tolist()

train_tokens = tokenizer(X_train_list, padding=True, truncation=True, return_tensors="pt")
val_tokens = tokenizer(X_val_list, padding=True, truncation=True, return_tensors="pt")
test_tokens = tokenizer(X_test_list, padding=True, truncation=True, return_tensors="pt")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




---

## 8. Dataset PyTorch


In [8]:
from torch.utils.data import Dataset
import torch

class TicketDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = TicketDataset(train_tokens, y_train.tolist())
val_dataset = TicketDataset(val_tokens, y_val.tolist())
test_dataset = TicketDataset(test_tokens, y_test.tolist())



---

## 9. Modèle BARThez pour classification


In [9]:
from transformers import AutoModelForSequenceClassification

n_classes = len(df_priorite['Classe'].unique())
model = AutoModelForSequenceClassification.from_pretrained("moussaKam/barthez", num_labels=n_classes)

Some weights of MBartForSequenceClassification were not initialized from the model checkpoint at moussaKam/barthez and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




---

## 10. Entraînement du modèle


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)

import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    preds = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

: 



---

## 11. Lancer l’entraînement


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: dembaseck1010 (dembaseck1010-universite-de-sherbrooke) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Voici comment améliorer la **Partie B** pour de meilleurs résultats sur la classification de la priorité, en suivant les mêmes principes que pour la Partie A.

---

### **4. Préparer les données pour la classification de priorité**

**Ajoute du code juste après la création de `df_priorite` pour nettoyer et enrichir les descriptions :**



In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# Nettoyage de la description et ajout du type de problème comme contexte
df_priorite['Description'] = (
    df['Description'].apply(clean_text) + " type: " + df['Type de problème'].str.lower()
)



---

### **6. Split train/val/test**

**Pour équilibrer les classes dans le jeu d'entraînement (oversampling) :**



In [ ]:
from sklearn.utils import resample

# Fusionne X_train et y_train pour suréchantillonner
df_train = pd.DataFrame({'Description': X_train, 'Classe': y_train})

# Trouve la taille de la classe majoritaire
max_size = df_train['Classe'].value_counts().max()

# Suréchantillonne chaque classe
lst = [df_train]
for class_index, group in df_train.groupby('Classe'):
    lst.append(group.sample(max_size - len(group), replace=True, random_state=42))
df_train_balanced = pd.concat(lst)

# Mélange
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42)

# Mets à jour X_train et y_train
X_train = df_train_balanced['Description']
y_train = df_train_balanced['Classe']



---

### **10. Entraînement du modèle**

**Ajoute l’early stopping pour éviter le surapprentissage :**



In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)



---

### **Conseils supplémentaires**

- Tu peux augmenter le nombre d’époques à 10 ou 15 dans `num_train_epochs` si tu utilises l’early stopping.
- Tu peux aussi essayer d’ajouter la colonne « Client » ou « Date » dans la description pour donner plus de contexte si tu le souhaites.

---

**Résumé :**
- Nettoie et enrichis les descriptions (section 4).
- Équilibre les classes dans le train (section 6).
- Ajoute l’early stopping (section 10).




---

## 12. Évaluer sur le jeu de test


In [ ]:
test_metrics = trainer.evaluate(test_dataset)
print(test_metrics)



---

## 13. Afficher les prédictions (optionnel)


In [ ]:
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
if isinstance(logits, tuple):
    logits = logits[0]
pred_labels = np.argmax(logits, axis=1)
true_labels = predictions.label_ids

for i in range(len(test_dataset)):
    print("Description:", X_test.iloc[i])
    print("True label:", encodeur_priorite.inverse_transform([true_labels[i]])[0])
    print("Predicted:", encodeur_priorite.inverse_transform([pred_labels[i]])[0])
    print("-" * 50)



---

## 14. Résultats (cellule texte)
Indiquez ici le score F1 obtenu sur le jeu de test.

---





## 15. Partie Bonus : Classification multi-tâches (type de problème + priorité)

---

### 15.1 Préparer les données multi-tâches



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Charger les données
df = pd.read_csv("IT_Support_Tickets_FR_200.csv")

# Nettoyage du texte
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df['Description'] = df['Description'].apply(clean_text)

# Encodage des deux cibles
encodeur_type = LabelEncoder()
encodeur_priorite = LabelEncoder()
df['Classe_type'] = encodeur_type.fit_transform(df['Type de problème'])
df['Classe_priorite'] = encodeur_priorite.fit_transform(df['Priorité'])



---

### 15.2 Split train/val/test



In [ ]:
from sklearn.model_selection import train_test_split

X = df['Description']
y_type = df['Classe_type']
y_priorite = df['Classe_priorite']

X_temp, X_test, y_type_temp, y_type_test, y_priorite_temp, y_priorite_test = train_test_split(
    X, y_type, y_priorite, test_size=0.2, random_state=42, stratify=y_type
)
X_train, X_val, y_type_train, y_type_val, y_priorite_train, y_priorite_val = train_test_split(
    X_temp, y_type_temp, y_priorite_temp, test_size=0.25, random_state=42, stratify=y_type_temp
)



---

### 15.3 Tokenizer BARThez



In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("moussaKam/barthez")
X_train_list = X_train.tolist()
X_val_list = X_val.tolist()
X_test_list = X_test.tolist()

train_tokens = tokenizer(X_train_list, padding=True, truncation=True, return_tensors="pt")
val_tokens = tokenizer(X_val_list, padding=True, truncation=True, return_tensors="pt")
test_tokens = tokenizer(X_test_list, padding=True, truncation=True, return_tensors="pt")



---

### 15.4 Dataset PyTorch multi-tâches



In [ ]:
from torch.utils.data import Dataset
import torch

class MultiTaskTicketDataset(Dataset):
    def __init__(self, encodings, labels_type, labels_priorite):
        self.encodings = encodings
        self.labels_type = labels_type
        self.labels_priorite = labels_priorite

    def __len__(self):
        return len(self.labels_type)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels_type"] = torch.tensor(self.labels_type[idx], dtype=torch.long)
        item["labels_priorite"] = torch.tensor(self.labels_priorite[idx], dtype=torch.long)
        return item

train_dataset = MultiTaskTicketDataset(train_tokens, y_type_train.tolist(), y_priorite_train.tolist())
val_dataset = MultiTaskTicketDataset(val_tokens, y_type_val.tolist(), y_priorite_val.tolist())
test_dataset = MultiTaskTicketDataset(test_tokens, y_type_test.tolist(), y_priorite_test.tolist())



---

### 15.5 Modèle multi-tâches (PyTorch)



In [ ]:
import torch.nn as nn
from transformers import AutoModel

class BarthezMultiTask(nn.Module):
    def __init__(self, model_name, num_labels_type, num_labels_priorite):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        self.classifier_type = nn.Linear(hidden_size, num_labels_type)
        self.classifier_priorite = nn.Linear(hidden_size, num_labels_priorite)

    def forward(self, input_ids, attention_mask=None, labels_type=None, labels_priorite=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        logits_type = self.classifier_type(pooled_output)
        logits_priorite = self.classifier_priorite(pooled_output)
        loss = None
        if labels_type is not None and labels_priorite is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits_type, labels_type) + loss_fct(logits_priorite, labels_priorite)
        return {"loss": loss, "logits_type": logits_type, "logits_priorite": logits_priorite}
        
num_labels_type = len(df['Classe_type'].unique())
num_labels_priorite = len(df['Classe_priorite'].unique())
model = BarthezMultiTask("moussaKam/barthez", num_labels_type, num_labels_priorite)



---

### 15.6 Adapter le Trainer HuggingFace



In [ ]:
from transformers import Trainer

def compute_metrics_multi(eval_pred):
    logits_type, logits_priorite = eval_pred.predictions
    labels_type, labels_priorite = eval_pred.label_ids
    preds_type = logits_type.argmax(axis=1)
    preds_priorite = logits_priorite.argmax(axis=1)
    from sklearn.metrics import f1_score, accuracy_score
    return {
        "f1_type": f1_score(labels_type, preds_type, average="weighted"),
        "f1_priorite": f1_score(labels_priorite, preds_priorite, average="weighted"),
        "acc_type": accuracy_score(labels_type, preds_type),
        "acc_priorite": accuracy_score(labels_priorite, preds_priorite),
    }

class MultiTaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels_type = inputs.pop("labels_type")
        labels_priorite = inputs.pop("labels_priorite")
        outputs = model(**inputs, labels_type=labels_type, labels_priorite=labels_priorite)
        loss = outputs["loss"]
        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        labels_type = inputs.pop("labels_type")
        labels_priorite = inputs.pop("labels_priorite")
        outputs = model(**inputs)
        return (outputs["loss"], (outputs["logits_type"].detach().cpu().numpy(), outputs["logits_priorite"].detach().cpu().numpy()), (labels_type.detach().cpu().numpy(), labels_priorite.detach().cpu().numpy()))

training_args = TrainingArguments(
    output_dir="./results_multitask",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs_multitask",
)

trainer = MultiTaskTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_multi,
)



---

### 15.7 Entraîner et évaluer



In [ ]:
trainer.train()
test_metrics = trainer.evaluate(test_dataset)
print(test_metrics)



---

### 15.8 Résultats

Dans une cellule texte, indique :

> **Score F1 type de problème :** ...  
> **Score F1 priorité :** ...  
> (Remplace par les valeurs de `test_metrics`.)

---
